In [ ]:
"""
This is the Pinsky and Rinzel Neuron model implementation in Brian Simulator. The model is stimulated using the frozen noise protocol. 
The input file used has a sampling rate of 20KHz and the simulation is run for 1 second. 
"""

In [ ]:
from brian2 import *
import matplotlib.pyplot as plt
import scipy.io as mat
import numpy as np

In [ ]:
#load the input file
I_theory = mat.loadmat("C:\\Users\\parih\\OneDrive\\Documents\\GitHub\\Microcircuit_Model\\input_inhibitory_tau_50_sampling_20_kHz.mat")["input_theory_i"]
hidden_state = mat.loadmat("C:\\Users\\parih\\OneDrive\\Documents\\GitHub\\Microcircuit_Model\\input_inhibitory_tau_50_sampling_20_kHz.mat")["hidden_state_i"]
I_scale = 20 #scaling factor for the input current - amplitude of the input current
I_baseline = -0. #baseline current - mean of the input current 
I_theory = I_theory.flatten()
Isyn_val = I_scale*I_theory+I_baseline
dt = 0.05*ms #timestep = 1/sampling rate - should stay consistent over the entire simulation
simtime = 1000*ms
Iapp = TimedArray(Isyn_val*uA, dt=dt)

In [ ]:
mean = np.mean(I_theory)
amplitude = np.max(I_theory) - np.min(I_theory)
print(mean,amplitude)

In [ ]:
#setting up the model
start_scope()
#parameters
pyr_parameters = {
"gLs" : 0.1*mS,
"gLd" : 0.1*mS,
"gNa" : 30.*mS,
"gKdr" : 15.*mS,
"gCa" : 10.*mS,
"gKahp" : 0.8*mS,
"gKC" : 15.*mS,
"VNa" : 60.0*mV,
"VCa" : 80.0*mV,
"VK" : -75.0*mV,
"VL" : -60.0*mV,
"Cm" : 3.0*uF,
"gc" : 2.1*mS,
# "gc" : 2.1mS, #for fig. 7: 3.0*mS, normal: 2.1*mS
"pp" : 0.5,
"alphac": 2.0/ms,
"betac" : 0.1/ms,
"betaqd": 0.001/ms, 
}

pyr_initialvalues = {
    'Vs' : -62.89223689*mV,
    'Vd' : -62.98248752*mV,
    'Cad' :0.21664282,
    'hs' : 0.99806345,
    'ns' :0.00068604,
    'sd' : 0.01086703,
    'cd' :0.00809387,
    'qd' : 0.0811213,
}

#equations for the neuron model using the orignal Pinsky and Rinzel equations and parameter values
eqs_pyr = '''
#membrane potential equations
#for soma
dVs/dt=(-ILs-INa-IK_DR+(gc/pp)*(Vd-Vs)+(Is/pp)-Isyns)/Cm: volt

#for dendrite
dVd/dt=(-ILd-ICad-IK_ahp-IK_C+((gc/(1.0-pp))*(Vs-Vd))+Id_FN/(1.0-pp)-Isynd)/Cm: volt  #-Isynd/(1.0-pp)


ILs   = gLs*(Vs-VL) : amp
INa   = gNa*Minfs*Minfs*hs*(Vs-VNa) :amp
IK_DR = gKdr*ns*(Vs-VK) :amp


ILd    = gLd*(Vd-VL) :amp
ICad   = gCa*sd*sd*(Vd-VCa) :amp
IK_ahp = gKahp*qd*(Vd-VK) :amp
IK_C   = gKC*cd*chid*(Vd-VK) :amp

#calcium concentration
dCad/dt=  (-0.13*ICad/uA-0.075*Cad)/ms: 1

# soma dynamics 
dhs/dt= alphahs-(alphahs+betahs)*hs: 1 
dns/dt= alphans-(alphans+betans)*ns: 1

# dendrite dynamics
dcd/dt= alphacd-(alphacd+betacd)*cd: 1 
dqd/dt= alphaqd-(alphaqd+betaqd)*qd: 1

# calcium dynamics
dsd/dt= alphasd-(alphasd+betasd)*sd: 1

# soma state

# sodium current
alphams=  0.32*(-46.9-Vs/mV)/(exp((-46.9-Vs/mV)/4.0)-1.0)/ms: Hz
betams=  0.28*(Vs/mV+19.9)/(exp((Vs/mV+19.9)/5.0)-1.0)/ms: Hz #orignal one
# betams=  0.28*(Vs/mV+70)/(exp((Vs/mV+70)/5.0)-1.0)/ms: Hz

Minfs=    alphams/(alphams+betams): 1 


alphahs=  0.128*exp((-43.0-Vs/mV)/18.0)/ms: Hz
betahs=   4.0/(1.0+exp((-20.0-Vs/mV)/5.0))/ms: Hz


# Kdr potassium current
alphans=  0.016*(-24.9-Vs/mV)/(exp((-24.9-Vs/mV)/5.0)-1.0)/ms: Hz
betans=   0.25*exp(-1.0-0.025*Vs/mV)/ms: Hz

# dendrite state


# calcium induced potassium current
alphacd=((int(Vd/mV<=-10)*exp(((Vd/mV+50.0)/11) - (Vd/mV + 53.3) / 27))/18.975 + (int(Vd/mV>-10)*2.0*exp((-53.5-Vd/mV)/27.0)))/ms : Hz
betacd=(int(Vd/mV<=-10)*(2.0*exp((-53.5-Vd/mV)/27.0)-alphacd*ms))/ms : Hz

# AHP: calcium induced potassium current
alphaqd=  clip(0.00002*Cad,0.00002*Cad, 0.01)/ms: Hz
betaqd = 0.001/ms: Hz
chid= clip(Cad/250.,Cad/250. , 1. ) : 1 

# calcium current
alphasd=  (1.6/(1.0+exp(-0.072*(Vd/mV-5.0))))/ms: Hz
betasd=   (0.02*(Vd/mV+8.9)/(exp((Vd/mV+8.9)/5.0)-1.0))/ms: Hz
# betasd= (0.02*(Vd/mV+13)/(exp((Vd/mV+13)/5.0)-1.0))/ms: Hz

Id_FN = Iapp(t) :amp #input current to the dendrite
# Is_FN = Iapp(t) :amp #input current to the soma : uncomment this for somatic input and change Is to Is_FN in the soma equation and Id_FN to Id in the dendrite equation
Is : amp
Id : amp
Isyns : amp
Isynd : amp
'''

#neuron groups
pyr_model = NeuronGroup(1, model=eqs_pyr, threshold='Minfs > 0.5', refractory=2 * ms,
                        dt=dt,method='euler',name='pyr',
                        namespace=pyr_parameters) 


#initial values
pyr_model.set_states(pyr_initialvalues)
pyr_model.Vs = pyr_initialvalues["Vs"]
pyr_model.Vd = pyr_initialvalues["Vd"]

#set monitor
Pyr = StateMonitor(pyr_model, ['Vs','Vd','Cad','ILs',"INa",'ICad','IK_ahp','IK_C','ILd','Id_FN','Is'], record=True)
S_pyr = SpikeMonitor(pyr_model)

run(simtime)

In [ ]:
fig,ax = plt.subplots(figsize=[20,5])
ax.plot(Pyr.t/ms, Pyr.Vs[0]/mV, label='Vs')
ax.plot(Pyr.t/ms, Pyr.Vd[0]/mV, label='Vd')
ax.set_xlabel('Time (ms)',fontsize=15)
ax.set_ylabel('Membrane Potential (mV)',fontsize=15)
ax1 = ax.twinx()
ax1.plot(Pyr.t/ms,Pyr.Cad[0],c='g',label='Cad')
ax.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Pyramidal Cell Id = -0.4',fontsize=20)
show()

In [ ]:
#plot the currents
fig = plt.figure(figsize=[20,10])
plt.subplot(5,1,1)
plt.plot(Pyr.t/ms,Pyr.ILs[0]/uA,label='ILs')
plt.ylabel('ILs (uA)',fontsize=15)
plt.legend(fontsize=15)
plt.title('Leak Current', fontsize = 12)
plt.subplot(5,1,2)
plt.plot(Pyr.t/ms,Pyr.INa[0]/uA,label='INa')
plt.ylabel('INa (uA)',fontsize=15)
plt.legend(fontsize=15)
plt.title('Sodium Current', fontsize = 12)
plt.subplot(5,1,3)
plt.plot(Pyr.t/ms,Pyr.ICad[0]/uA,label='ICad')
plt.ylabel('ICad (uA)',fontsize=15)
plt.legend(fontsize=15)
plt.title('Calcium Current', fontsize = 12)
plt.subplot(5,1,4)
plt.plot(Pyr.t/ms,Pyr.IK_ahp[0]/uA,label='IK_ahp')
plt.ylabel('IK_ahp (uA)',fontsize=15)
plt.legend(fontsize=15)
plt.xlabel('Time (ms)',fontsize=15)
plt.title('AHP Current', fontsize = 12)
plt.subplot(5,1,5)
plt.plot(Pyr.t/ms,Pyr.IK_C[0]/uA,label='IK_C')
plt.ylabel('IK_C (uA)',fontsize=15)
plt.legend(fontsize=15)
plt.xlabel('Time (ms)',fontsize=15)
plt.title('Potassium Current', fontsize = 12)
plt.show()

In [ ]:
#plot hidden states and input current
fig, ax = plt.subplots(figsize=(20,10))
ax2 = ax.twinx()
ax.plot(Pyr.t / ms, Pyr.Id_FN[0] / uA, label='Input Current', color='darkgrey')
ax2.plot(Pyr.t[:len(hidden_state)] / ms, hidden_state[:len(Pyr.t)] / uA, label='Hidden State', color='black')
ax.set_xlabel('Time (ms)', fontsize=20)
ax.set_ylabel('Input Current (uA)', fontsize=20)
ax2.set_ylabel('Hidden State', fontsize=20)
plt.title('Frozen Noise Input', fontsize=28)
ax.tick_params(axis='both', which='major', labelsize=20)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='lower right', fontsize=20)
ax2.tick_params(axis='both', which='major', labelsize=20)
plt.show()